In [ ]:
#run this code in GPU
#https://www.kaggle.com/datasets/ahemateja19bec1025/traffic-sign-dataset-classification (KAGGLE Site from where the data is)
#Connecting to Google drive because for inference images we've labelled the data in TEST folder
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#IMPORTING FILE FROM KAGGLE
#Go to Kaggle profice .... Scroll down ... There's API.....You'll get username and password from there
import os
os.environ['KAGGLE_USERNAME'] = "nikki2501" # username from the json file
#os.environ['KAGGLE_KEY'] = "18a26daea5b258038f4b2a03cdfee09f" # key from the json file

In [ ]:
#downloading Kaggle dataset
!kaggle datasets download -d ahemateja19bec1025/traffic-sign-dataset-classification

In [ ]:
# Unzip training data
from zipfile import ZipFile
file_name = "/content/traffic-sign-dataset-classification.zip"
with ZipFile(file_name, 'r') as zip:
  zip.extractall()
  print('done')

done


In [ ]:
pip install split-folders
#this library is used to divide data to traintest n val 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# from keras.optimizers import Adam
#IMPORTING necessary Libraries
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from keras.models import Sequential
from keras.models import Model
from keras.callbacks import EarlyStopping, ModelCheckpoint

import numpy as np
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Activation, Dense, Flatten, BatchNormalization, Conv2D, MaxPool2D,Dropout,GlobalAveragePooling2D
from keras.metrics import categorical_crossentropy
from keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import confusion_matrix
import itertools
import os
import shutil
import random
import glob
import matplotlib.pyplot as plt
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
# import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.image import imread
import pathlib
from sklearn.metrics import classification_report, confusion_matrix

# %matplotlib inline

In [ ]:
# SPLITTING DATAa
# import splitfolders
# # Split with a ratio.
# # To only split into training and validation set, set a tuple to `ratio`, i.e, `(.8, .2)`.
# splitfolders.ratio("/content/traffic_Data/DATA", output="FINAL",
#     # seed=1337, 
#     ratio=(.8, .2), group_prefix=None, move=False) # default values
# #.8 is taking 80% for train and .2 means taking 20% for validation

In [ ]:
#Copying path of drive .... saved the files in Google drive .... manually separated the inf data
train_dir ='/content/drive/MyDrive/Traffic_signal_/train'
test_dir = '/content/drive/MyDrive/Traffic_signal_/test'
inf_dir='/content/drive/MyDrive/Traffic_signal_/infer'

In [ ]:
#defining image size and batch size (batch size is )

img_width=331
img_height=331
batch_size=128

In [ ]:
#ImageDataGenerator is used Generate batches of tensor image data with real-time data augmentation.
#then here we are splitting the training img folder and separting 20% forvalidation data
train_datagen = ImageDataGenerator(rescale = 1./255,validation_split=0.2)
                                  #  rotation_range=30,
                                  #  zoom_range=0.4,
                                  #  horizontal_flip=True)

#all the above commented can be when we have to augment the images
#flow_from_directory Takes the path to a directory, and generates batches of augmented/normalized data
#to understand more about flow_from_directory refer this: https://studymachinelearning.com/keras-imagedatagenerator-with-flow_from_directory/
#
#class_mode : One of "categorical", "binary", "sparse", "input", or None. Default: "categorical".

train_generator = train_datagen.flow_from_directory(train_dir,
                                                    batch_size=batch_size,
                                                    subset="training",
                                                    class_mode='categorical',
                                                    target_size=(img_height, img_width))

Found 3926 images belonging to 58 classes.


In [ ]:
# val_datagen = ImageDataGenerator(rescale = 1./255)

val_generator = train_datagen.flow_from_directory(train_dir,
                                                 batch_size=batch_size,
                                                 class_mode='categorical',
                                                 subset="validation",
                                                 target_size=(img_height, img_width))

Found 956 images belonging to 58 classes.


In [ ]:
test_datagen = ImageDataGenerator(rescale = 1./255)#rescale = 1/255

test_generator = test_datagen.flow_from_directory(test_dir,
                                                  # batch_size=batch_size,
                                                  class_mode='categorical',
                                                  target_size=(img_height, img_width))

Found 652 images belonging to 58 classes.


In [ ]:
inf_datagen = ImageDataGenerator(rescale = 1./255)#rescale = 1/255

inf_generator = inf_datagen.flow_from_directory(inf_dir,
                                                  # batch_size=batch_size,
                                                  class_mode='categorical',
                                                  target_size=(img_height, img_width))

Found 515 images belonging to 58 classes.


In [ ]:
from keras.applications.nasnet import NASNetLarge

In [ ]:
input_shape=(331,331,3)

## **NasNetLarge**

Transfer learning is a machine learning method where a model developed for a task is reused as the starting point for a model on a second task.
using transfer learning.... imports model from it's stored location. no need to write each and every layer 


include_top false means because the fully connected layers at the end can only take fixed size inputs, which has been previously defined by the input shape and all processing in the convolutional layers. Any change to the input shape will change the shape of the input to the fully connected layers, making the weights incompatible (matrix sizes don't match and cannot be applied).


Imagenet weights is used for images classification,transfer learning model models use the imagenet weights

In [ ]:
#transfer learning used , explained above
base = NASNetLarge(input_shape,
                        include_top=False,
                        weights='imagenet')

#base.trainable = False

343610240/343610240 [==============================] - 2s 0us/step


In [ ]:
#Adding layers to our model, pooling it
#Then mentioning number of classes we have(58 in this dataset)

inputs = keras.Input(shape=(331,331,3))
# We make sure that the base_model is running in inference mode here,
# by passing `training=False`. This is important for fine-tuning, as you will
# learn in a few paragraphs.
x = base(inputs, training=False)
# Convert features of shape `base_model.output_shape[1:]` to vectors
x = keras.layers.GlobalAveragePooling2D()(x)
# A Dense classifier with a single unit (binary classification)
outputs = keras.layers.Dense(58)(x)
model = keras.Model(inputs, outputs)

In [ ]:
#Compiling the model 
#optimizer is used to reduce the losses
#different type of metric explained here :"https://keras.io/api/metrics/"
#Losses : The purpose of loss functions is to compute the quantity that a model should seek to minimize during training.
model.compile(optimizer=keras.optimizers.Adam(),
              loss=keras.losses.CategoricalCrossentropy(from_logits=True),#Computes the crossentropy loss between the labels and predictions.
              metrics=[keras.metrics.CategoricalAccuracy()])

In [ ]:
#give the summary about model...even the manually added layers and classes
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 331, 331, 3)]     0         
                                                                 
 NASNet (Functional)         (None, 11, 11, 4032)      84916818  
                                                                 
 global_average_pooling2d (G  (None, 4032)             0         
 lobalAveragePooling2D)                                          
                                                                 
 dense (Dense)               (None, 58)                233914    
                                                                 
Total params: 85,150,732
Trainable params: 233,914
Non-trainable params: 84,916,818
_________________________________________________________________


In [ ]:
#fitting the model on daatset and traing our machine, Train_gen and val_gen gives the accuracy by which it works and epochs means how many number of time it gonna run to model again and again
model.fit(train_generator, epochs=30, validation_data=val_generator)

Epoch 1/30
31/31 [==============================] - 1154s 36s/step - loss: 2.6281 - categorical_accuracy: 0.3943 - val_loss: 2.0031 - val_categorical_accuracy: 0.5429
Epoch 2/30
31/31 [==============================] - 150s 5s/step - loss: 1.3476 - categorical_accuracy: 0.7308 - val_loss: 1.3625 - val_categorical_accuracy: 0.6956
Epoch 3/30
31/31 [==============================] - 150s 5s/step - loss: 0.8987 - categorical_accuracy: 0.8362 - val_loss: 1.0437 - val_categorical_accuracy: 0.7646
Epoch 4/30
31/31 [==============================] - 150s 5s/step - loss: 0.6639 - categorical_accuracy: 0.8884 - val_loss: 0.8526 - val_categorical_accuracy: 0.8222
Epoch 5/30
31/31 [==============================] - 150s 5s/step - loss: 0.5174 - categorical_accuracy: 0.9233 - val_loss: 0.7030 - val_categorical_accuracy: 0.8734
Epoch 6/30
31/31 [==============================] - 150s 5s/step - loss: 0.4191 - categorical_accuracy: 0.9452 - val_loss: 0.5961 - val_categorical_accuracy: 0.8964
Epoch 7/

In [ ]:
#checking the accuracy of inference dataset
model.evaluate(inf_generator)

17/17 [==============================] - 116s 7s/step - loss: 0.3690 - categorical_accuracy: 0.9107


[0.3690038323402405, 0.9106796383857727]

In [ ]:
#checking the accuracy of test dataset
model.evaluate(test_generator)

21/21 [==============================] - 141s 7s/step - loss: 0.1048 - categorical_accuracy: 0.9877


[0.104767806828022, 0.987730085849762]

In [ ]:
#Saving model in .h5 file 
model.save('nn30_1.h5')

In [ ]:
# Have the streamlit code too 

# Streamlit use: Streamlit is an open-source app framework for Machine Learning and Data Science teams. Create beautiful web apps in minutes.

#run streamlit in your local host like VSCode or Jupyter

In [3]:
pip freeze > requirement.txt 

In [4]:
pip install -r requirements.txt

ERROR: Could not open requirements file: [Errno 2] No such file or directory: 'requirements.txt'


In [5]:
pip install pipreqs


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13723 sha256=dd2bf506672ea8d0f76b56d3f340092d442c03836d904c8b3c963ff4aed1e102
  Stored in directory: /root/.cache/pip/wheels/72/b0/3f/1d95f96ff986c7dfffe46ce2be4062f38ebd04b506c77c81b9
Successfully built docopt
